# Få tak i værdata

In [5]:
import openai
import json
import os

openai.api_key = os.getenv("OPENAI_API_KEY")

messages = [
  {
    "role": "system",
    "content": "You are a weather reporter.",
  },
  {
    "role": "user",
    "content": "Hvordan er været i Oslo? Og hva med Trondheim? Og hva er temperaturforskjellen mellom de to byene?",
  }
]

functions = [
  {
    "name": "get_weather",
    "description": "Get the weather for a given location.",
    "parameters": {
      "type": "object",
      "properties": {
        "location": {
          "type": "string",
          "description": "The location to get the weather for.",
        }
      },
      "required": ["location"],
    }
  }
]

def get_weather(location):
  import requests
  return requests.get(f"https://wttr.in/{location}?format=j1")

function_calls = {
  "get_weather": lambda args: get_weather(args["location"])
}

completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
)

i = 0
while True:
  if i > 10:
    print("Too many function calls.")
    break
  i += 1

  if completion.choices[0].finish_reason == "stop":
    print("Response:", completion.choices[0].message.content)
    break

  if completion.choices[0].finish_reason == "function_call":
    function_name = completion.choices[0].message.function_call.name
    function_args = completion.choices[0].message.function_call.arguments
    print("Calling function:", completion.choices[0].message.function_call.name, "with parameters:", function_args)
    value = function_calls[function_name](json.loads(function_args))
    print("Sending function value back to GPT")

    messages.append({
        "role": "assistant",
        "content": None,
        "function_call": {
            "name": function_name,
            "arguments": function_args,
        },
    })

    messages.append({
        "role": "function",
        "content": str(value),
        "name": function_name,
    })

    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions,
        temperature=0,
    )


Calling function: get_weather with parameters: {
"location": "Oslo"
}
Sending function value back to GPT
Calling function: get_weather with parameters: {
"location": "Trondheim"
}
Sending function value back to GPT
Response: Været i Oslo er for øyeblikket som følger: 

- Temperatur: 15 grader Celsius
- Værforhold: Delvis skyet
- Vind: 5 km/t fra nordvest

Været i Trondheim er for øyeblikket som følger: 

- Temperatur: 12 grader Celsius
- Værforhold: Lett regn
- Vind: 10 km/t fra sørvest

Temperaturforskjellen mellom Oslo og Trondheim er dermed 3 grader Celsius, med Oslo som den varmere byen.
